In [ ]:
import pandas as pd
import requests
import io

def fetch_and_clean_data():
    print("Initiating connection to NASA Exoplanet Archive...")
    
    # Switch to the standard API which hosts the legacy Kepler tables
    base_url = "https://exoplanetarchive.ipac.caltech.edu/cgi-bin/nstedAPI/nph-nstedAPI"
    
    # 1. We query the Kepler Stellar catalog
    print("Downloading stellar catalog (this might take a moment)...")
    stellar_params = {
        "table": "q1_q17_dr25_stellar",
        "select": "kepid,mass,feh,prov_sec",
        "format": "csv"
    }
    stellar_response = requests.get(base_url, params=stellar_params)
    
    # Check if the API returned an error
    if stellar_response.status_code != 200:
        raise ValueError(f"Stellar query failed: {stellar_response.text}")
        
    stellar_data = pd.read_csv(io.StringIO(stellar_response.text))
    # Rename the column here since the standard API doesn't support 'AS' in the select string
    stellar_data = stellar_data.rename(columns={"prov_sec": "age"})
    
    # 2. We query the Confirmed Planets/Candidates catalog (The sick people)
    print("Downloading planet host list...")
    koi_params = {
        "table": "q1_q17_dr25_koi",
        "select": "kepid",
        "where": "koi_disposition='CONFIRMED' or koi_disposition='CANDIDATE'",
        "format": "csv"
    }
    koi_response = requests.get(base_url, params=koi_params)
    
    # Check if the API returned an error
    if koi_response.status_code != 200:
        raise ValueError(f"KOI query failed: {koi_response.text}")
        
    koi_data = pd.read_csv(io.StringIO(koi_response.text))
    # Drop duplicates here since the standard API doesn't support 'DISTINCT'
    koi_data = koi_data.drop_duplicates(subset=['kepid'])
    
    # 3. We create our Binary Label (y = 1 or 0)
    # If the star's ID is in the koi_data, it gets a 1. Otherwise, 0.
    koi_data['has_planet'] = 1
    
    print("Merging datasets and creating binary labels...")
    master_data = pd.merge(stellar_data, koi_data, on='kepid', how='left')
    master_data['has_planet'] = master_data['has_planet'].fillna(0)
    
    # 4. The Clean-Up
    # Many stars have missing mass or age.
    initial_count = len(master_data)
    master_data = master_data.dropna(subset=['mass', 'feh', 'age'])
    final_count = len(master_data)
    
    print(f"Data cleaned! Dropped {initial_count - final_count} stars due to missing data.")
    print(f"Final dataset size: {final_count} stars ready for physics.")
    
    # Save to our computer
    master_data.to_csv("kepler_clean_demographics.csv", index=False)
    print("File saved as 'kepler_clean_demographics.csv'.")
    
    return master_data

if __name__ == "__main__":
    df = fetch_and_clean_data()
    print(df.head())

Initiating connection to NASA Exoplanet Archive...
Merging datasets and creating binary labels...
Data cleaned! Dropped 2942 stars due to missing data.
Final dataset size: 197096 stars ready for physics.
File saved as 'kepler_clean_demographics.csv'.
      kepid   mass   feh   age  has_planet
0  10000785  0.635 -1.00  DSEP         0.0
1  10000797  0.968 -0.44  DSEP         0.0
2  10000800  0.965 -0.04  DSEP         0.0
3  10000823  1.191 -0.16  DSEP         0.0
4  10000827  0.939 -0.10  DSEP         0.0
